Working on adding a lot of features just to see if it can get the score up regardless of how complicated or where the data is coming from

Features that I will be adding

* Taget given surface counts
* words from the target and source document
  * possible back prop into these vectors, idea is to replace tf-idf with some nn and back prop here
* the extra feature are going in with the target vec when approperate
    

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from theano import *
from lasagne.layers import InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np
from helpers import SimpleMaxingLayer, SimpleAverageLayer
from wordvecs import WordVectors, EmbeddingLayer, WordTokenizer
import json
import re
import random

theano.config.floatX = 'float32'
#theano.config.linker = 'cvm_nogc'
theano.config.openmp = True
theano.config.openmp_elemwise_minsize = 20000

Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
with open('/data/matthew/external-wiki2.json') as f:
    queries = json.load(f)['queries']

In [4]:
len(queries)

9915

In [5]:
sum([any([g['gold'] for g in v.values()]) for v in queries.values()])

8917

In [6]:
wordvectors = WordVectors(
    fname="/data/matthew/enwiki-20141208-pages-articles-multistream-links7-output1.bin",
    redir_fname='/data/matthew/enwiki-20141208-pages-articles-multistream-redirect7.json',
    negvectors=False,
    sentence_length=200,
)
wordvectors.add_unknown_words = False

In [7]:
# with open('/data/matthew/enwiki-20141208-pages-articles-multistream-redirects5.json') as f:
#     page_redirects = json.load(f)
page_redirects = wordvectors.redirects

In [8]:
len(wordvectors.vectors)

4056055

In [9]:
with open('/data/matthew/enwiki-20141208-pages-articles-multistream-surface-counts7.json') as f:
    surface_counts = json.load(f)

In [10]:
# try and make the surfaces items match what we are looking for
surface_counts_re = re.compile('([\.,!\?])')
for sk in surface_counts.keys():
    nsk = sk.replace('(', '-lrb-').replace(')', '-rrb-')
    nsk = surface_counts_re.sub(' \\1', nsk)
    if nsk != sk:
        surface_counts[nsk] = surface_counts[sk]

In [11]:
from wikireader import WikiRegexes, WikipediaReader

In [12]:
def PreProcessedQueries(wikipedia_dump_fname, vectors=wordvectors, queries=queries, redirects=page_redirects, surface=surface_counts):
    
    get_words = re.compile('[^a-zA-Z0-9 ]')
    get_link = re.compile('.*?\[(.*?)\].*?')
    
    wordvec = WordTokenizer(vectors, sentence_length=200)
    documentvec = WordTokenizer(vectors, sentence_length=1)
    
    queried_pages = set()
    for docs, q in queries.iteritems():
        wordvec.tokenize(docs)
        for sur, v in q.iteritems():
            wrds_sur = get_words.sub(' ', sur)
            wordvec.tokenize(wrds_sur)
            link_sur = get_link.match(sur).group(1)
            wordvec.tokenize(link_sur)
            for link in v['vals'].keys():
                wrds = get_words.sub(' ', link)
                wordvec.tokenize(wrds)
                tt = WikiRegexes.convertToTitle(link)
                documentvec.get_location(tt)
                queried_pages.add(tt)

    added_pages = set()
    for title in queried_pages:
        if title in redirects:
            #wordvec.tokenize(self.redirects[title])
            documentvec.get_location(redirects[title])
            added_pages.add(redirects[title])
    queried_pages |= added_pages

    page_content = {}

#     class GetWikipediaWords(WikipediaReader, WikiRegexes):

#         def readPage(ss, title, content):
#             tt = ss.convertToTitle(title)
#             if tt in queried_pages:
#                 cnt = ss._wikiToText(content)
#                 page_content[tt] = wordvec.tokenize(cnt)

#     GetWikipediaWords(wikipedia_dump_fname).read()
    
    rr = redirects
    rq = queried_pages
    rc = page_content
    rs = surface

    class PreProcessedQueriesCls(object):
        
        wordvecs = wordvec
        documentvecs = documentvec
        queries = queries
        redirects = rr
        queried_pages = rq
        page_content = rc
        surface_counts = rs
        
        
    return PreProcessedQueriesCls

In [13]:
basePreProcessedQueries = PreProcessedQueries('/data/matthew/enwiki-20141208-pages-articles-multistream.xml')

In [14]:
len(basePreProcessedQueries.wordvecs.reverse_word_location)

145739

In [15]:
len(basePreProcessedQueries.documentvecs.reverse_word_location)

443223

In [16]:
class EntityVectorLinkExp(basePreProcessedQueries):

    batch_size = 1000 #20000
    num_training_items = 500000 #200000

    def __init__(self):
        self.sentence_length = self.wordvecs.sentence_length
        self.document_length = 100
        self.num_words_to_use_conv = 5

        self._setup()


    def _setup(self):
        #self.x_document_input = T.imatrix('x_doc')

        #self.x_document_id = T.ivector('x_doc_id')
        self.x_surface_text_input = T.imatrix('x_surface_link')
        self.x_surface_context_input = T.imatrix('x_surface_cxt')  # TODO

        self.x_target_input = T.ivector('x_target')
        self.x_target_words = T.imatrix('x_target_words')
        self.x_matches_surface = T.ivector('x_match_surface')
        self.x_matches_counts = T.imatrix('x_matches_counts')
        self.x_link_id = T.ivector('x_link_id')

        #self.y_score = T.vector('y')
        self.y_answer = T.ivector('y_ans')  # contains the location of the gold answer so we can compute the loss
        self.y_grouping = T.imatrix('y_grouping')


        self.embedding_W = theano.shared(self.wordvecs.get_numpy_matrix().astype(theano.config.floatX))
        self.embedding_W_docs = theano.shared(self.documentvecs.get_numpy_matrix()).astype(theano.config.floatX)
        
#         self.document_l = lasagne.layers.InputLayer(
#             (None,self.document_length),
#             input_var=self.x_document_input
#         )

#         self.document_embedding_l = EmbeddingLayer(
#             self.document_l,
#             W=self.embedding_W,
#             add_word_params=False,
#         )

#         self.document_conv1_l = lasagne.layers.Conv2DLayer(
#             self.document_embedding_l,
#             num_filters=500,
#             filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
#             name='document_conv1',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.document_max_l = lasagne.layers.Pool2DLayer(
#             self.document_conv1_l,
#             name='document_pool1',
#             pool_size=(self.document_length - self.num_words_to_use_conv, 1),
#             mode='sum',
#         )

#         self.document_dens1 = lasagne.layers.DenseLayer(
#             self.document_max_l,
#             num_units=250,
#             name='doucment_dens1',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.document_drop1 = lasagne.layers.DropoutLayer(
#             self.document_dens1,
#             p=.25,
#         )

#         document_output_length = 200

#         self.document_dens2 = lasagne.layers.DenseLayer(
#             self.document_drop1,
#             num_units=225,
#             name='document_dens2',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.document_drop2 = lasagne.layers.DropoutLayer(
#             self.document_dens2,
#             p=.25,
#         )

#         self.document_dens3 = lasagne.layers.DenseLayer(
#             self.document_drop2,
#             num_units=document_output_length,
#             name='document_dens3',
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.document_output = lasagne.layers.get_output(self.document_dens3)

        self.surface_context_l = lasagne.layers.InputLayer(
            (None, self.sentence_length),
            input_var=self.x_surface_context_input,
        )

        self.surface_context_embedding_l = EmbeddingLayer(
            self.surface_context_l,
            W=self.embedding_W,
            add_word_params=False,
        )

        self.surface_context_conv1_l = lasagne.layers.Conv2DLayer(
            self.surface_context_embedding_l,
            num_filters=300,
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='surface_cxt_conv1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )

#         self.surface_context_avg1_l = SimpleAverageLayer(
#             [self.surface_context_conv1_l, self.surface_context_l],
#             #name='surface_context_avg'
#         )

        self.surface_context_pool1_l = lasagne.layers.Pool2DLayer(
            self.surface_context_conv1_l,
            name='surface_cxt_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='sum',
        )

        self.surface_input_l = lasagne.layers.InputLayer(
            (None, self.sentence_length),
            input_var=self.x_surface_text_input
        )

        self.surface_embedding_l = EmbeddingLayer(
            self.surface_input_l,
            W=self.embedding_W,
            add_word_params=False,
        )

        self.surface_conv1_l = lasagne.layers.Conv2DLayer(
            self.surface_embedding_l,
            num_filters=300,
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            name='surface_conv1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )

#         self.surface_avg1_l = SimpleAverageLayer(
#             [self.surface_conv1_l, self.surface_input_l],
#             #name='surface_avg'
#         )

        self.surface_pool1_l = lasagne.layers.Pool2DLayer(
            self.surface_conv1_l,
            name='surface_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='sum',
        )

        self.surface_merged_l = lasagne.layers.ConcatLayer(
            [self.surface_context_pool1_l, self.surface_pool1_l]
        )

        self.surface_dens1 = lasagne.layers.DenseLayer(
            self.surface_merged_l,
            name='surface_dens1',
            num_units=250,
            nonlinearity=lasagne.nonlinearities.tanh,
        )

#         self.surface_drop1 = lasagne.layers.DropoutLayer(
#             self.surface_dens1,
#             p=.25,
#         )

#         self.surface_dens2 = lasagne.layers.DenseLayer(
#             self.surface_drop1,
#             name='surface_dens2',
#             num_units=200,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.document_aligned_l = InputLayer(
#             (None, document_output_length),
#             input_var=self.document_output[self.x_document_id,:]
#         )

        ##############################################
        ## changed to not use the documented

#         self.source_l = lasagne.layers.ConcatLayer(
#             [self.document_aligned_l, self.surface_dens1]
#         )

        self.source_dens1 = lasagne.layers.DenseLayer(
            self.surface_dens1,   # CHANGED
            num_units=300,
            name='source_dens1',
            nonlinearity=lasagne.nonlinearities.tanh,
        )

        self.source_drop1 = lasagne.layers.DropoutLayer(
            self.source_dens1,
            p=.25,
        )

        self.source_dens12 = lasagne.layers.DenseLayer(
            self.source_drop1,
            num_units=250,
            name='source_dens12',
            nonlinearity=lasagne.nonlinearities.tanh,
        )

        self.source_drop12 = lasagne.layers.DropoutLayer(
            self.source_dens12,
            p=.25,
        )

        compared_vector_size = self.wordvecs.vector_size #+ 2 # extra space for if it matches the surface text

        self.source_dens2 = lasagne.layers.DenseLayer(
            self.source_drop12,
            num_units=compared_vector_size,  # this is the same size as the learned wikipedia vectors
            name='source_dens2',
            nonlinearity=lasagne.nonlinearities.linear,
        )

        self.source_out = lasagne.layers.get_output(self.source_dens2)

        matched_surface_reshaped = self.x_matches_surface.reshape(
            (self.x_matches_surface.shape[0], 1, 1, 1)).astype(theano.config.floatX)

        self.target_input_l = lasagne.layers.InputLayer(
            (None,),
            input_var=self.x_target_input
        )

        self.target_matched_surface_input_l = lasagne.layers.InputLayer(
            (None,1,1,1),
            input_var=matched_surface_reshaped,
        )
        
        self.target_matched_counts_input_l = lasagne.layers.InputLayer(
            (None,4),
            input_var=self.x_matches_counts.astype(theano.config.floatX),
        )

        self.target_embedding_l = EmbeddingLayer(
            lasagne.layers.reshape(self.target_input_l, ([0], 1)),
            W=self.embedding_W_docs,
            add_word_params=False,
        )

        self.target_combined_feats_l = lasagne.layers.ConcatLayer(
            [self.target_embedding_l, self.target_matched_surface_input_l,
            lasagne.layers.reshape(self.target_matched_counts_input_l, ([0],1,1,[1]))],
            axis=3
        )

        self.target_words_input_l = lasagne.layers.InputLayer(
            (None,self.sentence_length),
            input_var=self.x_target_words,
        )

        self.target_words_embedding_l = EmbeddingLayer(
            self.target_words_input_l,
            W=self.embedding_W,
            add_word_params=False,
        )

        self.target_words_conv1_l = lasagne.layers.Conv2DLayer(
            self.target_words_embedding_l,
            name='target_wrds_conv1',
            filter_size=(self.num_words_to_use_conv, self.wordvecs.vector_size),
            num_filters=350,
            nonlinearity=lasagne.nonlinearities.tanh,
        )

        self.target_words_pool1_l = lasagne.layers.Pool2DLayer(
            self.target_words_conv1_l,
            name='target_wrds_pool1',
            pool_size=(self.sentence_length - self.num_words_to_use_conv, 1),
            mode='sum',
        )

        self.target_merge_l = lasagne.layers.ConcatLayer(
            [lasagne.layers.reshape(self.target_words_pool1_l, ([0], [1])),
             lasagne.layers.reshape(self.target_embedding_l, ([0], [3]))]
        )

        self.target_dens1 = lasagne.layers.DenseLayer(
            self.target_merge_l,
            name='target_wrds_dens1',
            num_units=400,
            nonlinearity=lasagne.nonlinearities.tanh,
        )

        self.target_drop1 = lasagne.layers.DropoutLayer(
            self.target_dens1,
            p=.25,
        )

        self.target_dens2 = lasagne.layers.DenseLayer(
            self.target_drop1,
            name='target_wrds_dens1',
            num_units=compared_vector_size,
            nonlinearity=lasagne.nonlinearities.linear,
        )

        self.target_simple = lasagne.layers.DenseLayer(
            self.target_combined_feats_l,
            name='target_simple1',
            num_units=compared_vector_size,
            nonlinearity=lasagne.nonlinearities.linear,
        )

#         self.target_dens1 = lasagne.layers.DenseLayer(
#             self.target_conv1_l,
#             name='target_dens1',
#             num_units=300,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )

#         self.target_drop1 = lasagne.layers.DropoutLayer(
#             self.target_dens1,
#             p=.25,
#         )

#         self.target_dens2 = lasagne.layers.DenseLayer(
#             self.target_drop1,
#             name='target_dens2',
#             num_units=300,
#             nonlinearity=lasagne.nonlinearities.tanh,
#         )



        #self.target_out = lasagne.layers.get_output(self.target_embedding_l)


#         self.target_out = T.concatenate(
#             [self.embedding_W[self.x_target_input],
#              matched_surface_reshaped,
#             1-matched_surface_reshaped],
#              axis=1)


        #self.target_out = self.embedding_W[self.x_target_input]
        #self.target_out = lasagne.layers.get_output(self.target_dens2)

        self.target_out = lasagne.layers.get_output(self.target_simple)

        # compute the cosine distance between the two layers
        self.source_aligned_l = self.source_out[self.x_link_id, :]

        # this uses scan internally, which means that it comes back into python code to run the loop.....fml
        self.dotted_vectors =  T.batched_dot(self.target_out, self.source_aligned_l)
        # diag also does not support a C version.........
        #self.dotted_vectors = T.dot(self.target_out, self.source_aligned_l.T).diagonal()

        def augNorm(v):
            return T.maximum(T.basic.pow(T.basic.pow(T.basic.abs_(v), 2).sum(axis=1) + .001, .5), .001)

        self.res_l = self.dotted_vectors / (augNorm(self.target_out) * augNorm(self.source_aligned_l) + .001)
#         self.res_l = self.dotted_vectors / ((self.target_out.norm(1, axis=1) + .001) *
#                                             (self.source_aligned_l.norm(1, axis=1) + .001))

        self.res_cap = T.clip((T.tanh(self.res_l) + 1) / 2, .001, .999)

        #self.golds = self.res_cap[self.y_answer]

#         def maxOverRange(indx):
#             #return T.max(self.res_cap[T.arange(indx[0],indx[1])]) - self.res_cap[indx[2]]
#             #return -( self.res_l[indx[2]] - T.log(T.exp(self.res_l[T.arange(indx[0],indx[1])]).sum()) )
#             return -( self.res_l[indx[2]] - self.res_l[indx[0]])

#         # build a tensor to make a matrix with one set on each dimention
#         self.grouped, grouped_update = theano.scan(maxOverRange, sequences=self.y_grouping)

        def setSubSelector(indx, outputs):
            return T.set_subtensor(outputs[T.arange(indx[0], indx[1]), indx[3]], 1)

        num_target_samples = self.res_l.shape[0]

        select_seq = T.concatenate([
            self.y_grouping,
            T.arange(self.y_grouping.shape[0]).reshape((self.y_grouping.shape[0], 1))
        ], axis=1)

        self.selecting_matrix, _ = theano.scan(
            setSubSelector,
            outputs_info=T.zeros((num_target_samples, num_target_samples)),
            #n_steps=self.y_grouping.shape[0]
            sequences=select_seq,
        )

        self.groupped_elems = T.dot(self.selecting_matrix[-1], T.diag(T.exp(self.res_l)))
        self.groupped_res = T.log(self.groupped_elems.sum(axis=0)[T.arange(self.y_grouping.shape[0])])
        self.loss_vec = self.groupped_res - self.res_l[self.y_grouping[:,2]]

        self.all_params = (
            #lasagne.layers.get_all_params(self.target_dens2) +
            # TODO: add params for the target stuff,
            lasagne.layers.get_all_params(self.target_simple) +
            lasagne.layers.get_all_params(self.source_dens2)
            #lasagne.layers.get_all_params(self.document_dens2)
        )

        # weight the positive samples more since there are fewer of them,
        # freaking hack
        #self.loss_vec = -(10 * self.y_score * T.log(self.res_cap) + (1.0 - self.y_score) * T.log(1.0 - self.res_cap))

        #self.loss_vec = T.nnet.binary_crossentropy(self.res_cap, self.y_score)

        #self.loss_vec = T.exp(T.max(self.res_cap - self.res_cap[self.y_answer] + .1, 0)) - 1  # TODO: maybe have some squared term here or something?

        # this one works reasonably well
        #self.loss_vec = - T.log((T.clip(self.res_cap[self.y_answer] - self.res_cap, -1.0, 0.4) + 1.0) / 1.5)

        #self.loss_vec = self.grouped

        #self.loss_vec = - T.log((T.clip(self.res_l[self.y_answer] - self.res_l, -40.0, 10.0) + 40.0) / 51.0)
        #self.loss_vec = T.max(self.res_l[self.y_answer] - self.res_l + .1, 0)

        self.updates = lasagne.updates.adadelta(self.loss_vec.mean(), self.all_params)

        self.func_inputs = [
            #self.x_document_input,
            self.x_surface_text_input, self.x_surface_context_input, #self.x_document_id,
            self.x_target_input, self.x_matches_surface, self.x_matches_counts, self.x_link_id,
            self.y_answer, self.y_grouping
        ]  # self.x_target_words,

        self.train_func = theano.function(
            self.func_inputs,
            [self.res_cap, self.loss_vec.sum(), self.loss_vec],
            updates=self.updates,
            on_unused_input='ignore',
        )

        self.test_func = theano.function(
            self.func_inputs,
            [self.res_cap, self.loss_vec.sum(), self.loss_vec],
            on_unused_input='ignore',
        )

    def reset_accums(self):
        self.current_documents = []
        self.current_surface_context = []
        self.current_surface_link = []
        self.current_link_id = []
        self.current_target_input = []
        self.current_target_words = []
        self.current_target_matches_surface = []
        self.current_target_id = []
        self.current_target_goal = []
        self.current_learning_groups = []
        self.learning_targets = []
        self.current_surface_target_counts = []
        
        
            
    def compute_batch(self, isTraining=True, useTrainingFunc=True):
        if isTraining and useTrainingFunc:
            func = self.train_func
        else:
            func = self.test_func
        self.reset_accums()
        self.total_links = 0
        self.total_loss = 0.0
        
        self.failed_match = []
        self.failed_page_match = []

        get_words = re.compile('[^a-zA-Z0-9 ]')
        get_link = re.compile('.*?\[(.*?)\].*?')

        for doc, queries in self.queries.iteritems():
            # skip the testing documents while training and vice versa
            if queries.values()[0]['training'] != isTraining:
                continue
            docid = len(self.current_documents)
            self.current_documents.append(self.wordvecs.tokenize(doc, length=self.document_length))
            for surtxt, targets in queries.iteritems():
                self.current_link_id.append(docid)
                surid = len(self.current_surface_link)
                self.current_surface_context.append(self.wordvecs.tokenize(get_words.sub(' ' , surtxt)))
                surlink = get_link.match(surtxt).group(1)
                self.current_surface_link.append(self.wordvecs.tokenize(surlink))
                surmatch = surlink.lower()
                surcounts = self.surface_counts.get(surmatch)
                if not surcounts:
                    self.failed_match.append(surmatch)
                    surcounts = {}
                #target_page_input = []
                target_words_input = []
                target_matches_surface = []
                target_inputs = []
                target_learings = []
                target_match_counts = []
                target_gold_loc = -1
                target_group_start = len(self.current_target_input)
                for target in set(targets['vals'].keys() + random.sample(self.documentvecs.reverse_word_location, 1)) - {None,}:
                    # skip the items that we don't know the gold for
                    if not targets['gold'] and isTraining:
                        continue
                    isGold = target == targets['gold']
                    #cnt = self.page_content.get(WikiRegexes.convertToTitle(target))
                    wiki_title = WikiRegexes.convertToTitle(target)
                    cnt = self.documentvecs.get_location(wiki_title)
                    if cnt is None or wiki_title == 'nil':
                        # were not able to find this wikipedia document
                        # so just ignore tihs result since trying to train on it will cause
                        # issues
                        # 
                        # there are also nil queries that are generated for every document
                        # but we actually have a nil page that is getting referenced
                        # so just filter it out for now
                        continue
                    if isGold:
                        target_gold_loc = len(target_inputs)
                    #target_page_input.append(cnt)
                    target_words_input.append(self.wordvecs.tokenize(get_words.sub(' ', target)))
                    target_inputs.append(cnt)  # page_content already tokenized
                    target_matches_surface.append(int(surmatch == target.lower()))
                    target_learings.append((targets, target))
                    tmc = surcounts.get(wiki_title, 0)
                    if tmc is 0:
                        self.failed_page_match.append((surcounts, wiki_title))
                    target_match_counts.append(tmc)
                    #if wiki_title not in surcounts:
                    #    print surcounts, wiki_title
                if target_gold_loc is not -1 or not isTraining:  # if we can't get the gold item
                    # contain the index of the gold item for these items, so it can be less then it
                    gold_loc = (len(self.current_target_goal) + target_gold_loc)
                    sorted_match_counts = [-4,-3,-2,-1] + sorted(set(target_match_counts))
                    #print sorted_match_counts
                    target_match_counts_indicators = [
                        [
                            int(s == sorted_match_counts[-1]),
                            int(s == sorted_match_counts[-2]),
                            int(s == sorted_match_counts[-3]),
                            int(s <= sorted_match_counts[-4]),
                        ]
                        for s in target_match_counts
                    ]
                    self.current_target_goal += [gold_loc] * len(target_inputs)
                    self.current_target_input += target_inputs
                    self.current_target_id += [surid] * len(target_inputs)
                    self.current_target_words += target_words_input   # TODO: add
                    self.current_target_matches_surface += target_matches_surface
                    self.current_surface_target_counts += target_match_counts_indicators
                    target_group_end = len(self.current_target_input)
                    self.current_learning_groups.append(
                        [target_group_start, target_group_end,
                         gold_loc])

                #self.current_target_goal.append(isGold)
                self.learning_targets += target_learings
            if len(self.current_target_id) > self.batch_size:
                self.run_batch(func)
                if self.total_links > self.num_training_items:
                    return self.total_loss / self.total_links

        if len(self.current_target_id) > 0:
            self.run_batch(func)

        return self.total_loss / self.total_links

    def run_batch(self, func):
        res_vec, loss_sum, loss_vec = func(
            #self.current_documents,
            self.current_surface_link, self.current_surface_context, #self.current_link_id,
            self.current_target_input, self.current_target_matches_surface, self.current_surface_target_counts, self.current_target_id,
            self.current_target_goal, self.current_learning_groups,
            # self.current_target_words,
        )
        self.check_params()
        self.total_links += len(self.current_target_id)
        self.total_loss += loss_sum
        for i in xrange(len(res_vec)):
            # save the results from this pass
            l = self.learning_targets[i]
            if l[1] in l[0]['vals']:
                l[0]['vals'][ l[1] ] = float(res_vec[i])
        self.reset_accums()

    def check_params(self):
        if any([np.isnan(v.get_value(borrow=True)).any() for v in self.all_params]):
            raise RuntimeError('nan in some of the parameters')



queries_exp = EntityVectorLinkExp()

/home/matthew/.virtualenvs/nlp-convnet/lib/python2.7/site-packages/theano/scan_module/scan_perform_ext.py:135: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility
  from scan_perform.scan_perform import *


In [17]:
def evalCurrentState(trainingData=True, numSamples=50000):
    all_measured = 0
    all_correct = 0
    all_trained = 0
    for qu in queries.values():
        for en in qu.values():
            if en['training'] != trainingData:
                continue
            for e in en:
                if en['gold']:
                    if all_trained > numSamples:
                        break
                    all_measured += 1
                    all_trained += len(en['vals'].values())
                    m = max(en['vals'].values())
                    if en['vals'][en['gold']] == m and m != 0:
                        all_correct += 1
           
    r = all_measured, float(all_correct) / all_measured
    print r
    return r

In [18]:
import random
def augmentTrainingData():
    for quk in queries.keys():
        qu = queries[quk]
        for enk in qu.keys():
            en = qu[enk]
            if not en['gold']:
                del qu[enk]
        if not qu:
            del queries[quk]
    for qu in queries.values():
        training = random.random() > .15
        for en in qu.values():
            en['training'] = training
augmentTrainingData()

In [31]:
def findWrongItems(trainingData=True, numSamples=50):
    ret = {}
    for qu in queries.values():
        for ek, en in qu.items():
            if en['training'] != trainingData:
                continue
            for e in en:
                if en['gold']:
                    if len(ret) > numSamples:
                        return ret
                    m = max(en['vals'].values())
                    if en['vals'][en['gold']] != m and m != 0:
                        ret[ek] = en
    return ret

In [19]:
queries_exp.check_params()

In [20]:
queries_exp.num_training_items = 50000

In [17]:
%time print queries_exp.compute_batch()

0.174694755739
CPU times: user 5min 1s, sys: 1.62 s, total: 5min 3s
Wall time: 41.3 s


In [79]:
len(queries_exp.current_surface_target_counts)

1192

In [ ]:
evalCurrentState(False, 500000)

In [ ]:
evalCurrentState(True, 500000)

In [21]:
queries_exp.embedding_W_docs.get_value(borrow=True).shape, len(queries_exp.documentvecs.reverse_word_location)

((443223, 300), 443223)

In [22]:
queries_exp.embedding_W_docs.get_value(borrow=True)[0,:]

array([  1.33422855e-03,   1.47313438e-03,  -1.27675373e-03,
        -1.09268189e-03,   4.55525704e-04,   1.00702920e-03,
         3.13618977e-04,   7.04447448e-05,  -1.20117189e-03,
         7.39491777e-04,  -1.45212805e-03,   4.16514085e-04,
        -2.50447600e-04,  -3.19061277e-04,  -1.10524497e-03,
        -6.27390516e-04,   8.59781925e-04,   1.00845331e-03,
         9.89685068e-04,   5.32480888e-04,   5.14933257e-04,
        -1.26754760e-03,  -1.36525475e-03,   1.65654498e-03,
         1.26708986e-03,   1.03001914e-03,  -2.01314295e-04,
         1.33855187e-03,  -1.64998375e-04,   2.44903553e-04,
        -4.97436522e-05,  -9.94415255e-04,   4.37418610e-04,
        -4.45810962e-04,  -1.27512612e-03,   1.58472697e-03,
         1.45975745e-03,  -3.65041109e-04,  -7.53784188e-05,
         1.69728592e-04,  -1.21256511e-03,  -1.33570354e-03,
         1.51824951e-03,   2.10062663e-05,  -8.60799162e-04,
        -1.01404823e-03,  -1.02549233e-03,   5.65643306e-04,
         6.71386733e-05,

In [23]:
exp_results = []

for i in xrange(10):
    res = (i, queries_exp.compute_batch())
    print res
    exp_results.append(res)
exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('training state', evalCurrentState(True, queries_exp.num_training_items)))
exp_results.append(('testing state', evalCurrentState(False, queries_exp.num_training_items)))

(0, 0.16735710548942262)
(1, 0.15028690041663562)
(2, 0.1397832968675268)
(3, 0.13201666452661467)
(4, 0.12662432220768347)
(5, 0.12132103437932863)
(6, 0.11739539470759805)
(7, 0.11375313768138517)
(8, 0.10989574552932949)
(9, 0.10665113175751745)
(2879, 0.3501215699895797)
(2936, 0.5657356948228883)


In [34]:
surface_counts['1970']['1970']

779

In [32]:
findWrongItems()

{u"'' Pachinko Sexy Reaction '' is an ecchi [arcade game] -LRB- more adult than '' Pachinko Sexy Reaction 2 ''": {u'gold': u'Arcade game',
  u'training': True,
  u'vals': {u'-NIL-': 0,
   u'Arcade game': 0.2620583176612854,
   u"Bucky O'Hare (arcade game)": 0.14432048797607422,
   u'Donkey Kong (video game)': 0.373124361038208,
   u'Game': 0.1474006175994873,
   u'Game (2011 film)': 0.15339088439941406,
   u'Game (food)': 0.49143701791763306,
   u'Game (hunting)': 0.5533944368362427,
   u'Game (rapper)': 0.20140263438224792,
   u'Game (retailer)': 0.23536133766174316,
   u'Ninja Gaiden (arcade)': 0.373124361038208,
   u'Pac-Man': 0.16817247867584229,
   u'Popeye (video game)': 0.14348697662353516,
   u'Star Wars (1983 video game)': 0.16195404529571533,
   u'Teenage Mutant Ninja Turtles (arcade game)': 0.7568063735961914,
   u'The Game (rapper)': 0.189531147480011,
   u'Video game': 0.8709296584129333,
   u'XXNILXX': 0,
   u'arcade game': 0.1378268003463745,
   u'game': 0.55339443683624

In [25]:
queries.values()[0]

{u', 1921 August 8 , 1984 -RRB- , born in [Philadelphia] , was an American television and motion picture actor .': {u'gold': u'Philadelphia',
  u'training': True,
  u'vals': {u'-NIL-': 0,
   u'2014\u201315 Philadelphia Flyers season': 0,
   u'Advanta Championships of Philadelphia': 0.1833834946155548,
   u'Ala\u015fehir': 0.23164361715316772,
   u'Philadelphia': 0.8694509267807007,
   u'Philadelphia (film)': 0.40520280599594116,
   u'Philadelphia (magazine)': 0.5469392538070679,
   u'Philadelphia 76ers': 0.6569768786430359,
   u'Philadelphia County, Pennsylvania': 0.36047476530075073,
   u'Philadelphia Eagles': 0.8495123386383057,
   u'Philadelphia Flyers': 0.5137590169906616,
   u'Philadelphia International Airport': 0.3388158679008484,
   u'Philadelphia Phantoms': 0.16947954893112183,
   u'Philadelphia Phillies': 0.8689995408058167,
   u'Philadelphia Union': 0.138202965259552,
   u'Philadelphia, Mississippi': 0.6829430460929871,
   u'Philadelphia, Pennsylvania': 0.8625656366348267,
 

In [21]:


for i in xrange(5):
    res = (i, queries_exp.compute_batch())
    print res
    exp_results.append(res)
exp_results.append(('testing run', queries_exp.compute_batch(False)))
exp_results.append(('training state', evalCurrentState(True, queries_exp.num_training_items)))
exp_results.append(('testing state', evalCurrentState(False, queries_exp.num_training_items)))

(0, 0.12797744133809885)
(1, 0.12219550112039526)


IndexError: index 443222 is out of bounds for size 443222
Apply node that caused the error: AdvancedSubtensor1(<TensorType(float32, matrix)>, Flatten{1}.0)
Toposort index: 79
Inputs types: [TensorType(float32, matrix), TensorType(int32, vector)]
Inputs shapes: [(443222, 300), (2165,)]
Inputs strides: [(1200, 4), (4,)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Reshape{4}(AdvancedSubtensor1.0, MakeVector{dtype='int64'}.0)]]

HINT: Re-running with most Theano optimization disabled could give you a back-trace of when this node was created. This can be done with by setting the Theano flag 'optimizer=fast_compile'. If that does not work, Theano optimizations can be disabled with 'optimizer=None'.
HINT: Use the Theano flag 'exception_verbosity=high' for a debugprint and storage map footprint of this apply node.

In [26]:
443222 in queries_exp.current_target_input

True

In [28]:
len(queries_exp.documentvecs.reverse_word_location)

443223

In [31]:
queries_exp.documentvecs.reverse_word_location[0]

In [48]:
exp_results

[(0, 0.13412095478504968),
 (1, 0.08384761093081565),
 (2, 0.075165537455837886),
 (3, 0.074463398532355854),
 (4, 0.074139625877423787),
 ('testing run', 0.077500098314125229),
 ('training state', (28229, 0.39268128520315987)),
 ('testing state', (28313, 0.7650902412319429))]

In [32]:
[(p, np.isnan(p.get_value(borrow=True)).any()) for p in queries_exp.all_params]

[(target_simple1.W, True),
 (target_simple1.b, True),
 (surface_cxt_conv1.W, True),
 (surface_cxt_conv1.b, True),
 (surface_conv1.W, True),
 (surface_conv1.b, True),
 (surface_dens1.W, True),
 (surface_dens1.b, True),
 (source_dens1.W, True),
 (source_dens1.b, True),
 (source_dens12.W, True),
 (source_dens12.b, True),
 (source_dens2.W, True),
 (source_dens2.b, True)]

In [18]:
queries_exp.failed_match

[u'pachinko sexy reaction 2',
 u'16th-century',
 u'santiago ram n y cajal ,',
 u'abul-faraj ibn al-jawzi',
 u'abul-faraj ibn al-jawzi',
 u'abu muhammad ibn hazm',
 u"hmcs '' cayuga ''",
 u"hmcs '' athabaskan ''",
 u"seychellois creole -lrb- '' kreol",
 u'33 bce',
 u'dactylic hexameters',
 u"'' zembla ''",
 u'rebecca ray',
 u'weekending',
 u'weekending',
 u'edward wiley']